# Quick start with pytorch [link](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

PyTorch has two primitives to work with data: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from tqdm import tqdm

/home/abdullah/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch offers domain-specific libraries such as [TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and [TorchAudio](https://pytorch.org/audio/stable/index.html),
all of which include datasets. For this tutorial, we  will be using a TorchVision dataset.

The ``torchvision.datasets`` module contains ``Dataset`` objects for many real-world vision data like
CIFAR, COCO ([full list here](https://pytorch.org/vision/stable/datasets.html)). In this tutorial, we
use the FashionMNIST dataset. Every TorchVision ``Dataset`` includes two arguments: ``transform`` and
``target_transform`` to modify the samples and labels respectively.



In [11]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [40]:
# loading the data into the dataloader 

BATCH_SIZE = 64 

train_dataloader = DataLoader(training_data, batch_size = BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size = BATCH_SIZE)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
y[0]

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


tensor(9)

In [41]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
print(f'using device: {device}')

using device: cuda


In [50]:
# building the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    # forward is part of __init__ not the class itself
    def forward(self, x):
        x  = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [51]:
# loss and optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [52]:
# single epoch loop

def train_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    # putting model in training mode
    model.train()
    
    for batch, (x, y) in enumerate(dataloader): # The enumerate() function adds a counter to an iterable also start and stop.
        
        x = x.to(device)
        y = y.to(device)
        
        # computing the loss
        pred = model(x)
        loss = loss_fn(pred, y)
        
        
        # computing gradients
        optimizer.zero_grad()
        loss.backward() # how loss  is back ??? (implicitly by pred and y variables)
        optimizer.step()
        
        # printing every 100 batch
        if batch %100==0:
            loss = loss.item()
            current = (batch+1) * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [53]:
# testing function

def test(dataloader, model, loss_fn):
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    #puting model in evaluation stage
    model.eval()
    
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)
            
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss = test_loss / num_batches
    correct /= size
    print(f'Test error \n Accuracy: {correct:>4f}  Avg loss: {test_loss:>8f} \n')
            

In [54]:
# main training loop
EPOCHS = 5

for t in range(EPOCHS):
    print(f'Epoch: {t+1}/{EPOCHS} -------------------------------')
    train_epoch(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print('Done !')
    

Epoch: 1/5 -------------------------------
loss: 2.310085  [   64/60000]
loss: 2.296363  [ 6464/60000]
loss: 2.279557  [12864/60000]
loss: 2.278315  [19264/60000]
loss: 2.262035  [25664/60000]
loss: 2.236336  [32064/60000]
loss: 2.242483  [38464/60000]
loss: 2.211272  [44864/60000]
loss: 2.206981  [51264/60000]
loss: 2.182131  [57664/60000]
Test error 
 Accuracy: 0.491500  Avg loss: 2.173853 

Epoch: 2/5 -------------------------------
loss: 2.177288  [   64/60000]
loss: 2.168404  [ 6464/60000]
loss: 2.112413  [12864/60000]
loss: 2.136609  [19264/60000]
loss: 2.091968  [25664/60000]
loss: 2.035027  [32064/60000]
loss: 2.067522  [38464/60000]
loss: 1.990804  [44864/60000]
loss: 1.990203  [51264/60000]
loss: 1.930049  [57664/60000]
Test error 
 Accuracy: 0.595100  Avg loss: 1.919119 

Epoch: 3/5 -------------------------------
loss: 1.942335  [   64/60000]
loss: 1.915231  [ 6464/60000]
loss: 1.792647  [12864/60000]
loss: 1.845326  [19264/60000]
loss: 1.747152  [25664/60000]
loss: 1.68885

In [57]:
# saveing the model
torch.save(model.state_dict(), 'data/model.pth')

In [59]:
# loading the saved model
saved_model = NeuralNetwork()
saved_model.load_state_dict(torch.load('data/model.pth'))

<All keys matched successfully>

In [65]:
x, y = test_data[0]
y

9

In [68]:
# using the saved model

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0]
with torch.no_grad():
    pred = saved_model(x)
    print(pred.shape)
    predicted, actual = classes[pred[0].argmax(0)], classes[y] # across dim 0
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

torch.Size([1, 10])
Predicted: "Ankle boot", Actual: "Ankle boot"
